<img src="./assets/chicago_thin.gif" style="float:center ; margin: 0px ; width:1000px;"> 

# Predicting West Nile Virus cases with Data Science

---

### Finding/Fighting the Virus
The West Nile Virus(WNV) is a scourge plaguing Chicago. In 2018, there were 176 reported cases and 17 resultings deaths from the virus. 

Using Data analysis and machine learning classifiers like Random Forests and Logistic Regression, we seek to find out the most important factors (eg. Wind, temperature, pesticide use, etc.) that determine the presence of WNV in any given district in the Windy City. 


### Background
West Nile virus is transmitted through the bite of various subspecies of the Culex mosquito. Common symptoms include fever, nausea, headache and muscle aches. Symptoms may last from a few days to a few weeks. However, four out of five people infected with West Nile virus will not show any symptoms. In rare cases, severe illness including meningitis, or even death, can occur. 

### Datasets
We have four different data sets that we're working with for this project:

**train.csv & test.csv** - These datasets contain information from several checks on different neighbourhoods in Chicago(10506 and 116293 respectively). The latter dataset is does not contain information about whether the cases are WNV cases or not. This will be the data set used to validate our model. The results will be submitted to a Kaggle competition. 

**spray.csv** - This dataset contains time and location details of 14835 pesticide spraying operations all over Chicago city. 

**weather.csv** - As the proliferation of mosquitos has to do with weather conditions, we will also be using data from a detailed dataset containing information of 2944 days from 2007 to 2014.

```
Project 4: Predicting West Nile Virus cases with Datascience
|__ Cleaning.ipynb   
|__ EDA.ipynb
|__ Modelling.ipynb  
|__ data
|   |__ train.csv
|   |__ test.csv
|   |__ spray.csv
|   |__ weather.csv
|__ images
|   |__ (various_images_used_for_design_purposes)
|__ README.md
```

### Sources

Kaggle competition details and the datasets can be found [here](https://www.kaggle.com/c/predict-west-nile-virus/). Kaggle's public leaderboard uses roughly 30% of the dataset to score an AUC (Area Under Curve) metric [here](https://www.kaggle.com/wiki/AreaUnderCurve).


# 1. Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import random
import time
%matplotlib inline

### 1.1 Cleaning the Weather dataset

In [2]:
weather=pd.read_csv('data/weather.csv')

In [3]:
weather

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,2,2014-10-29,49,40,45,M,34,42,20,0,...,,M,M,M,0.00,29.42,30.07,8.5,29,9.0
2940,1,2014-10-30,51,32,42,-4,34,40,23,0,...,,0,M,0.0,0.00,29.34,30.09,5.1,24,5.5
2941,2,2014-10-30,53,37,45,M,35,42,20,0,...,RA,M,M,M,T,29.41,30.10,5.9,23,6.5
2942,1,2014-10-31,47,33,40,-6,25,33,25,0,...,RA SN,0,M,0.1,0.03,29.49,30.20,22.6,34,22.9


---
**The Problem with Ms and NaNs** -
According to our data dictionary for the weather dataset (which you can check via the *noaa_weather_qclcd_documentation.pdf* file on the main folder), Missing Data is filled in with an M instead of the usual NaN. Let's start by dealing with this problem. 

In [4]:
weather.replace('M',np.nan,inplace=True)

In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2933 non-null   object 
 5   Depart       1472 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2940 non-null   object 
 8   Heat         2933 non-null   object 
 9   Cool         2933 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        1472 non-null   object 
 14  Water1       0 non-null      float64
 15  SnowFall     1472 non-null   object 
 16  PrecipTotal  2942 non-null   object 
 17  StnPressure  2940 non-null   object 
 18  SeaLevel     2935 non-null   object 
 19  Result

In [6]:
# Decision: drop columns with 40% missing data 
# (also because these factors doesnt seem to affect mozzy breeding)
# isolating columns we don't want
dropthese=weather.columns[weather.isnull().sum()>(len(weather)*0.4)]
dropthese

Index(['Depart', 'Depth', 'Water1', 'SnowFall'], dtype='object')

# Comment: but... how did you know that these factors don't seem to affect mozzy breeding?

In [7]:
#dropping these columns
weather.drop(dropthese,axis=1,inplace=True)

---
**Dashes?** -
There is also the case of odd entries with dashes in the Sunrise and Sunset columns. 

In [8]:
#Checking on the value_counts in Sunset and Sunrise
weather["Sunset"].value_counts().head()

-       1472
1931      96
1930      56
1929      48
1928      32
Name: Sunset, dtype: int64

In [9]:
#Checking on the value_counts in Sunset and Sunrise
weather["Sunrise"].value_counts().head()

-       1472
0416     104
0417      64
0419      40
0420      32
Name: Sunrise, dtype: int64

In [10]:
# potentially, details about sunset, sunrise matters  
# We will impute with NaNs ahead of our EDA. 
# Note that we are not using zeroes because we might need to transform the data 
weather.replace('-',np.nan,inplace=True)

---
**Typecasting** -
There is a fair amount of typecasting work that needs to be done for the weather dataset.

In [11]:
# Checking out the column types 
weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

In [12]:
# changing date to datetime object
weather['Date']=pd.to_datetime(weather['Date'])

In [13]:
#isolating columns listed as "object" columns
needcast=weather.columns[weather.dtypes=='object']
needcast

Index(['Tavg', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'AvgSpeed'],
      dtype='object')

In [14]:
#Checking unique values in all our columns listed as "object" columns
[print(weather[i].value_counts()) for i in needcast]
# With one exception, Most of our columns are visually int/floats but the computer is reading them as objects. 
# Note: The exception is CodeSum, which is not int/floats but must rmb not to drop when 
# make dummies since i choose to leave space in

73    138
70    117
77    117
75    110
71    109
74    107
72    104
69    103
78    102
76    100
68     99
79     98
66     93
67     89
61     88
64     86
80     84
65     84
63     81
57     67
62     66
60     61
50     57
81     55
58     49
53     49
82     48
55     48
54     48
56     46
52     46
59     45
51     36
83     34
49     29
45     28
46     24
47     24
84     21
44     19
48     17
86     16
85     16
42     15
43     12
87      9
41      7
40      5
91      4
88      4
89      4
39      4
36      2
90      2
37      2
38      2
94      1
93      1
92      1
Name: Tavg, dtype: int64
63    135
65    131
59    129
61    123
64    121
62    118
67    117
66    113
60    111
69    107
57    105
70    102
71     99
55     99
68     98
58     95
56     95
52     78
46     72
72     71
53     69
47     61
51     57
54     56
73     56
50     55
45     53
48     51
49     50
74     50
44     45
42     35
40     31
43     31
75     20
41     19
38     17
39     15
36   

[None, None, None, None, None, None, None, None, None, None, None]

In [15]:
# The is also the case of a "T" value in the 'PrecipTotal' total column
# A check with a data dictionary shows that T represents "Trace" amounts lower than 0.01
# Let's replace with 0.005 since smallest observation in the col before zero is 0.01
weather['PrecipTotal'].replace('  T',0.005,inplace=True)

In [16]:
# Dropping "CodeSum" from the isolated list
needcast_dropCodeSum=needcast.drop('CodeSum')

In [17]:
# Typecasting remaining columns to floats
weather.loc[:,needcast_dropCodeSum]=weather[needcast_dropCodeSum].astype('float')

In [18]:
# Checking dtypes for all columns
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                  float64
DewPoint                int64
WetBulb               float64
Heat                  float64
Cool                  float64
Sunrise               float64
Sunset                float64
CodeSum                object
PrecipTotal           float64
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir               int64
AvgSpeed              float64
dtype: object

---
**Checking for remaining null values** -
There are still some nulls left to be dealt with. 

In [19]:
# Checking for nulls 
weather.isnull().sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg             11
DewPoint          0
WetBulb           4
Heat             11
Cool             11
Sunrise        1472
Sunset         1472
CodeSum           0
PrecipTotal       2
StnPressure       4
SeaLevel          9
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

In [20]:
# assumming that temperature is normally distributed, we can put those null values with the average
null_Tavg=weather[weather['Tavg'].isnull()]
weather.loc[null_Tavg.index,'Tavg']=(null_Tavg['Tmax']-null_Tavg['Tmin'])/2

In [21]:
# Checking for nulls
weather.isnull().sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg              0
DewPoint          0
WetBulb           4
Heat             11
Cool             11
Sunrise        1472
Sunset         1472
CodeSum           0
PrecipTotal       2
StnPressure       4
SeaLevel          9
ResultSpeed       0
ResultDir         0
AvgSpeed          3
dtype: int64

In [22]:
# Checking on the dates when "WetBulb" has null values
weather.isnull().sum()
weather.Date[weather['WetBulb'].isnull()]

848    2009-06-26
2410   2013-08-10
2412   2013-08-11
2415   2013-08-12
Name: Date, dtype: datetime64[ns]

In [23]:
# Decision: we can impute these values with those on the same day but different station...
# ...since this 2 stations are just 30mins drive away based on google maps
bad_days=weather[weather['WetBulb'].isnull()].Date

for date in bad_days:
    print(weather[weather.Date==date]['WetBulb'].dropna().values,date)
    # since we only have 2 stations, we can just use replace the value for both dates

[67.] 2009-06-26 00:00:00
[63.] 2013-08-10 00:00:00
[64.] 2013-08-11 00:00:00
[68.] 2013-08-12 00:00:00


In [24]:
#Imputation
for date in bad_days:
    weather.loc[weather['Date']==date,'WetBulb']=weather[weather.Date==date]['WetBulb'].dropna().values

In [25]:
check=['848','2410','2412','2415']
weather.iloc[848]
#Imputation succeeded

Station                          1
Date           2009-06-26 00:00:00
Tmax                            86
Tmin                            69
Tavg                            78
DewPoint                        60
WetBulb                         67
Heat                             0
Cool                            13
Sunrise                        418
Sunset                        1931
CodeSum                           
PrecipTotal                      0
StnPressure                    NaN
SeaLevel                     29.85
ResultSpeed                    6.4
ResultDir                        4
AvgSpeed                       8.2
Name: 848, dtype: object

In [26]:
#Creating a list of columns with null values
bad_feature=['Heat','Cool','PrecipTotal','SeaLevel','AvgSpeed']

In [27]:
#Creating a nested for loop to replace null values with values from the alternate station
for i in bad_feature:
    bad_days=weather[weather[i].isnull()].Date
    for date in bad_days:
        weather.loc[weather['Date']==date,i]=weather[weather.Date==date][i].dropna().values

In [28]:
# But...we have a slight problem
# StnPressure has a date with where both station is NaN
weather[weather['StnPressure'].isnull()]

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
87,2,2007-06-13,86,68,77.0,53,62.0,0.0,12.0,NaN,NaN,,0.0,NaN,30.09,7.0,5,8.6
848,1,2009-06-26,86,69,78.0,60,67.0,0.0,13.0,418.0,1931.0,,0.0,NaN,29.85,6.4,4,8.2
2410,1,2013-08-10,81,64,73.0,57,63.0,0.0,8.0,454.0,1900.0,,0.0,NaN,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,75.0,55,63.0,0.0,10.0,NaN,NaN,,0.0,NaN,30.07,6.0,6,7.4


In [29]:
# Impute the ones where there are values from the other station to use
for date in ['2007-06-13','2009-06-26']:
    weather.loc[weather['Date']==date,'StnPressure']=weather[weather.Date==date]['StnPressure'].dropna().values

In [30]:
# Decision: for that day where both stations had null values for StnPressure...
# ...we will first find the mean value from the days before and after...
weather[(weather['Date']=='2013-08-11')|(weather['Date']=='2013-08-11')]['StnPressure'].mean()

29.385

In [31]:
# And impute the values. 
weather.loc[2410:2411,'StnPressure']=29.385

In [32]:
# Checking on null values. 
weather.isnull().sum()

Station           0
Date              0
Tmax              0
Tmin              0
Tavg              0
DewPoint          0
WetBulb           0
Heat              0
Cool              0
Sunrise        1472
Sunset         1472
CodeSum           0
PrecipTotal       0
StnPressure       0
SeaLevel          0
ResultSpeed       0
ResultDir         0
AvgSpeed          0
dtype: int64

In [33]:
# Checking to see if there are duplicate entries
weather.duplicated().sum()

0

In [34]:
# Saving a cleaned copy of the dataset
weather.to_csv('data/weather_cleaned.csv' , index=False)

## 1.2 Cleaning the Spray Dataset

In [35]:
# Reading in the spray data set
spray=pd.read_csv('data/spray.csv')

In [36]:
# Checking for nulls
spray.info()
# There are only nulls from the "Time" column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       14835 non-null  object 
 1   Time       14251 non-null  object 
 2   Latitude   14835 non-null  float64
 3   Longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [37]:
# Checking in on those null "Time" entries
spray[spray['Time'].isnull()]
# The nulls are seem to be from the same date.. but Latitude and Longitude is all different

,Date,Time,Latitude,Longitude
1030,2011-09-07,NaN,41.987092,-87.794286
1031,2011-09-07,NaN,41.987620,-87.794382
1032,2011-09-07,NaN,41.988004,-87.794574
1033,2011-09-07,NaN,41.988292,-87.795486
1034,2011-09-07,NaN,41.988100,-87.796014
...,...,...,...,...
1609,2011-09-07,NaN,41.995876,-87.811615
1610,2011-09-07,NaN,41.995972,-87.810271
1611,2011-09-07,NaN,41.995684,-87.810319
1612,2011-09-07,NaN,41.994724,-87.810415


In [38]:
# Checking whether our guess is correct
spray[spray['Time'].isnull()]['Date'].nunique()

1

---
**A large-scale spraying operation with bad data entry?** - Potentially, there could have been a large scale operation of mozzy spraying on this date and thus officials could not assign an appropriate time to impute for all the different sprays at once. We can make this inference with some confidence because the langitude and longitude are different. We'll make a decision to impute these observations with a fair average timing. 

In [39]:
# Checking what is the earliest time a spraying operation occurs
pd.to_datetime(spray['Time']).min()

Timestamp('2020-03-29 00:00:01')

In [40]:
# Checking what is the latest time a spraying operation occurs
pd.to_datetime(spray['Time']).max()

Timestamp('2020-03-29 23:59:58')

In [41]:
# Checking out operations that happen in the morning. 
spray['Time'].str.contains('AM').mean()
# Only one percent of operations happen in the morning. 
# sprays are done in the evenings mostly

0.010244895095081048

In [42]:
# Taking a random sample from the "Time" column
rand_time=spray['Time'].sample(1,random_state=36)
rand_time
# we set boundaries for our random samples 1.5hr before and after this time

10164    8:49:15 PM
Name: Time, dtype: object

In [43]:
# Creating random timings within the boundaries
start_timestamp = time.mktime(time.strptime('Sep 7 2011  6:37:27', '%b %d %Y %I:%M:%S'))
end_timestamp = time.mktime(time.strptime('Sep 7 2011  9:37:38', '%b %d %Y %I:%M:%S'))
random.seed(36)
# Imputing Nulls with our random timings
for i in range(1030,1613+1):
    spray.loc[i,'Time']=time.strftime('%I:%M:%S', time.localtime(random.randrange(start_timestamp,end_timestamp)))+' PM'
    print(spray.loc[i,'Time'])

08:07:12 PM
06:53:23 PM
06:43:08 PM
07:55:07 PM
06:58:54 PM
06:37:33 PM
08:54:40 PM
09:28:39 PM
07:25:38 PM
07:45:04 PM
07:51:01 PM
09:35:35 PM
08:35:31 PM
08:16:59 PM
09:10:23 PM
07:50:39 PM
09:05:39 PM
06:58:40 PM
09:14:46 PM
08:28:49 PM
08:51:15 PM
08:23:56 PM
08:11:48 PM
08:25:29 PM
07:43:53 PM
08:24:30 PM
07:23:13 PM
07:11:09 PM
08:57:50 PM
08:53:16 PM
07:33:01 PM
09:00:35 PM
07:57:49 PM
07:27:49 PM
08:51:43 PM
09:19:27 PM
08:06:04 PM
09:07:25 PM
07:23:27 PM
06:38:18 PM
09:19:37 PM
08:22:42 PM
08:52:30 PM
07:15:18 PM
07:36:52 PM
08:31:29 PM
08:18:59 PM
07:34:17 PM
08:52:30 PM
07:47:24 PM
09:24:21 PM
06:47:25 PM
07:41:55 PM
09:28:35 PM
08:55:57 PM
08:43:41 PM
09:34:52 PM
08:56:06 PM
07:58:16 PM
08:41:03 PM
07:00:31 PM
07:41:34 PM
07:59:52 PM
07:32:56 PM
09:12:29 PM
07:23:11 PM
07:25:55 PM
06:54:50 PM
07:42:53 PM
08:19:31 PM
08:35:53 PM
09:06:15 PM
06:41:52 PM
09:28:03 PM
08:21:20 PM
08:12:30 PM
08:50:56 PM
07:23:54 PM
09:03:53 PM
09:11:22 PM
07:46:43 PM
07:00:40 PM
07:04:27 PM
09:3

In [44]:
# Checking for any remaining nulls
spray.isnull().sum()

Date         0
Time         0
Latitude     0
Longitude    0
dtype: int64

In [45]:
# Checking for duplicates 
spray[spray.duplicated()]

,Date,Time,Latitude,Longitude
485,2011-09-07,7:43:40 PM,41.983917,-87.793088
490,2011-09-07,7:44:32 PM,41.986460,-87.794225
491,2011-09-07,7:44:32 PM,41.986460,-87.794225
492,2011-09-07,7:44:32 PM,41.986460,-87.794225
493,2011-09-07,7:44:32 PM,41.986460,-87.794225
...,...,...,...,...
1025,2011-09-07,7:44:32 PM,41.986460,-87.794225
1026,2011-09-07,7:44:32 PM,41.986460,-87.794225
1027,2011-09-07,7:44:32 PM,41.986460,-87.794225
1028,2011-09-07,7:44:32 PM,41.986460,-87.794225


In [46]:
# Inspecting the Duplicates
spray[spray.Date=='2011-09-07'].Time.value_counts()

7:44:32 PM     541
08:29:45 PM      2
07:50:04 PM      2
06:44:22 PM      2
08:01:25 PM      2
              ... 
8:04:32 PM       1
9:00:57 PM       1
8:11:40 PM       1
8:57:42 PM       1
07:00:40 PM      1
Name: Time, Length: 1559, dtype: int64

In [47]:
# The duplicates are obviously a typo.. the timings have such small value_counts
# Decision: Let's drop them
spray.drop_duplicates(inplace=True)
# Saving to csv
spray.to_csv('data/spray_cleaned.csv', index=False)

## 1.3 Cleaning the Train dataset

In [48]:
#Reading in the Train Dataset
train=pd.read_csv('data/train.csv',parse_dates=['Date'])

In [49]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [50]:
#Checking for null values
train.info()
#Set looks very much intact

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    10506 non-null  datetime64[ns]
 1   Address                 10506 non-null  object        
 2   Species                 10506 non-null  object        
 3   Block                   10506 non-null  int64         
 4   Street                  10506 non-null  object        
 5   Trap                    10506 non-null  object        
 6   AddressNumberAndStreet  10506 non-null  object        
 7   Latitude                10506 non-null  float64       
 8   Longitude               10506 non-null  float64       
 9   AddressAccuracy         10506 non-null  int64         
 10  NumMosquitos            10506 non-null  int64         
 11  WnvPresent              10506 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), ob

In [51]:
# There is a fair amount(813!) of duplicate rows
train[train.duplicated()]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
99,2007-06-26,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,1,0
295,2007-07-11,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,50,0
351,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0
353,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0
512,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",CULEX RESTUANS,33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",41.883284,-87.705085,8,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
10013,2013-08-29,"7100 North Harlem Avenue, Chicago, IL 60631, USA",CULEX PIPIENS/RESTUANS,71,N HARLEM AVE,T233,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,50,1
10102,2013-09-06,"3700 South Pulaski Road, Chicago, IL 60623, USA",CULEX PIPIENS,37,S PULASKI RD,T063,"3700 S PULASKI RD, Chicago, IL",41.825610,-87.726549,9,9,0
10124,2013-09-06,"ORD Terminal 5, O'Hare International Airport, ...",CULEX PIPIENS/RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,9,50,1
10262,2013-09-12,"ORD Terminal 5, O'Hare International Airport, ...",CULEX PIPIENS/RESTUANS,10,W OHARE AIRPORT,T900,"1000 W OHARE AIRPORT, Chicago, IL",41.974689,-87.890615,9,50,0


In [52]:
#Checking out the duplicates on block 22
#Decision: We are not dropping the duplicates because...
#from block 22, we can see that there are multiple checks on a single day..
#...and thus we should leave this row alone
train[(train['Date']=='2007-06-26')&(train['Block']==22)]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
98,2007-06-26,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,1,0
99,2007-06-26,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,1,0
100,2007-06-26,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,2,0
101,2007-06-26,"2200 West 89th Street, Chicago, IL 60643, USA",CULEX RESTUANS,22,W 89TH ST,T096,"2200 W 89TH ST, Chicago, IL",41.731922,-87.677512,8,5,0


## 1.4 Cleaning the Test Dataset

In [53]:
#Checking out the entries in the test set.
test=pd.read_csv('data/test.csv',parse_dates=['Date'])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Id                      116293 non-null  int64         
 1   Date                    116293 non-null  datetime64[ns]
 2   Address                 116293 non-null  object        
 3   Species                 116293 non-null  object        
 4   Block                   116293 non-null  int64         
 5   Street                  116293 non-null  object        
 6   Trap                    116293 non-null  object        
 7   AddressNumberAndStreet  116293 non-null  object        
 8   Latitude                116293 non-null  float64       
 9   Longitude               116293 non-null  float64       
 10  AddressAccuracy         116293 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 9.8+ MB


In [54]:
#no duplicates here.
test[test.duplicated()]

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy


> **Moving on!** - We have done initial checks on our data and cleaned both our weather and spray datasets. We will now move on to EDA. 

In [55]:
#To start the EDA process, call up the CSV files via the following codelines:

# test=pd.read_csv('data/test.csv',parse_dates=['Date'])
# train=pd.read_csv('data/train.csv',parse_dates=['Date'])
# spray_cleaned=pd.read_csv('data/spray_cleaned.csv')
# weather_cleaned=pd.read_csv('data/weather_cleaned.csv.csv')
